In [ ]:
# prompt: No module named 'whisper'

!pip install git+https://github.com/openai/whisper.git


In [ ]:
# prompt: No module named 'pydub'

!pip install pydub


In [ ]:
!pip install --upgrade pyannote.audio

In [ ]:

!pip install datasets

In [ ]:
from pyannote.audio import Pipeline

diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization@2.1", use_auth_token='HF_TOKEN'
)

config.yaml:   0%|          | 0.00/500 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

config.yaml:   0%|          | 0.00/318 [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1+cu121. Bad things might happen unless you revert torch to 1.x.


...

embedding_model.ckpt:   0%|          | 0.00/83.3M [00:00<?, ?B/s]

mean_var_norm_emb.ckpt:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

classifier.ckpt:   0%|          | 0.00/5.53M [00:00<?, ?B/s]

label_encoder.txt:   0%|          | 0.00/129k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/speechbrain/utils/checkpoints.py:194: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(path, map_location=device

In [8]:
from datasets import load_dataset

concatenated_librispeech = load_dataset(
    "sanchit-gandhi/concatenated_librispeech", split="train", streaming=True
)
sample = next(iter(concatenated_librispeech))
print(sample)

{'audio': {'path': 'processed.wav', 'array': array([ 0.        ,  0.        ,  0.        , ..., -0.00097656,
       -0.00109863, -0.00146484]), 'sampling_rate': 16000}}


In [7]:
from pydub import AudioSegment
from IPython.display import Audio
import numpy as np

# Ses dosyası yolunu belirtin (örneğin, "path/to/your/audio_file.mp3")
audio_file_path = "/content/recording_2.mp3"

# Ses dosyasını yükle
audio = AudioSegment.from_file(audio_file_path)

# Ses dosyasını NumPy dizisine dönüştür
audio_array = np.array(audio.get_array_of_samples())

# Eğer ses stereo ise (iki kanal) kanalları ayır
if audio.channels == 2:
    audio_array = audio_array.reshape((-1, 2))  # İki kanalı ayır
    audio_array = audio_array.mean(axis=1)  # Her iki kanalı ortalayıp tek kanala dönüştür
elif audio.channels == 1:
    audio_array = audio_array  # Tek kanalı doğrudan al
else:
    raise ValueError("Beklenmeyen kanal sayısı")

# Ses frekansını alın
sampling_rate = audio.frame_rate
print(sampling_rate)

# IPython'da sesi dinlemek için Audio nesnesi oluştur
Audio(audio_array.astype(np.float32), rate=sampling_rate)






Output hidden; open in https://colab.research.google.com to view.

In [16]:
from IPython.display import Audio

Audio(audio_array["audio"]["array"], rate=audio_array["audio"]["sampling_rate"])

In [ ]:
from pydub import AudioSegment
from IPython.display import Audio
import numpy as np
import torch
import json

# Ses dosyası yolunu belirtin
audio_file_path = "/content/recording_2.mp3"

# Ses dosyasını yükle
audio = AudioSegment.from_file(audio_file_path)

# Ses dosyasını NumPy dizisine dönüştür
audio_array = np.array(audio.get_array_of_samples())

# Eğer ses stereo ya da üç kanallı ise
if audio.channels == 2:
    audio_array = audio_array.reshape((-1, 2))  # İki kanalı ayır
    audio_array = audio_array.mean(axis=1)  # Her iki kanalı ortalayıp tek kanala dönüştür
elif audio.channels == 1:
    audio_array = audio_array  # Tek kanalı doğrudan al
elif audio.channels == 3:
    audio_array = audio_array.reshape((-1, 3))  # Üç kanalı ayır
    audio_array = audio_array.mean(axis=1)  # Üç kanalı ortalayıp tek kanala dönüştür
else:
    raise ValueError("Beklenmeyen kanal sayısı")

# Ses frekansını alın
sampling_rate = audio.frame_rate

# Ses kaydını tensor'a dönüştür
input_tensor = torch.from_numpy(audio_array[None, :]).float()

# Diyalizasyondan çıktıyı al
outputs = diarization_pipeline(
    {"waveform": input_tensor, "sample_rate": sampling_rate}
)

# Annotation nesnesinin içeriklerini almak
segments = []
for turn, _, speaker in outputs.itertracks(yield_label=True):
    segment = {
        "start": turn.start,
        "end": turn.end,
        "speaker": speaker,  # Speaker label'ını burada kullanabilirsiniz
        "text": turn.text  # Konuşma metnini ekliyoruz
    }
    segments.append(segment)

# JSON formatına dönüştürmek için
output_json = json.dumps(segments, indent=4)

# Çıktıyı yazdır
print(output_json)

# IPython'da sesi dinlemek için Audio nesnesi oluştur
Audio(audio_array.astype(np.float32), rate=sampling_rate)


In [28]:
import torch

input_tensor = torch.from_numpy(sample["audio"]["array"][None, :]).float()
outputs = diarization_pipeline(
    {"waveform": input_tensor, "sample_rate": sample["audio"]["sampling_rate"]}
)

# Annotation nesnesinin içeriklerini almak
segments = []
for turn, _, speaker in outputs.itertracks(yield_label=True):
    segment = {
        "start": turn.start,
        "end": turn.end,
        "speaker": speaker  # Speaker label'ını burada kullanabilirsiniz
    }
    segments.append(segment)

# JSON formatına dönüştürmek için
import json
output_json = json.dumps(segments, indent=4)

print(output_json)



[
    {
        "start": 0.03096875,
        "end": 14.543468750000002,
        "speaker": "SPEAKER_01"
    },
    {
        "start": 15.387218750000002,
        "end": 21.25971875,
        "speaker": "SPEAKER_00"
    }
]


In [ ]:
from transformers import pipeline

asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-base",
)


asr_pipeline(
    sample["audio"].copy(),
    generate_kwargs={"max_new_tokens": 256},
    return_timestamps=True,
)


from pyannote.audio import Pipeline

diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization@2.1", use_auth_token='HF_TOKEN'
)

In [ ]:
asr_pipeline(
    sample["audio"].copy(),
    generate_kwargs={"max_new_tokens": 256},
    return_timestamps=True,
)

In [ ]:
from pyannote.audio import Pipeline

diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization@2.1", use_auth_token='HF_TOKEN'
)

In [ ]:
!pip install git+https://github.com/huggingface/speechbox

In [ ]:
from speechbox import ASRDiarizationPipeline

pipeline = ASRDiarizationPipeline(
    asr_pipeline=asr_pipeline, diarization_pipeline=diarization_pipeline
)

In [33]:
pipeline(sample["audio"].copy())

[{'speaker': 'SPEAKER_01',
  'text': ' The second and importance is as follows. Sovereignty may be defined to be the right of making laws. In France, the king really exercises a portion of the sovereign power, since the laws have no weight.',
  'timestamp': (0.0, 15.48)},
 {'speaker': 'SPEAKER_00',
  'text': " He was in a favored state of mind, owing to the blight his wife's action threatened to cast upon his entire future.",
  'timestamp': (15.48, 21.28)}]

In [34]:
def tuple_to_string(start_end_tuple, ndigits=1):
    return str((round(start_end_tuple[0], ndigits), round(start_end_tuple[1], ndigits)))


def format_as_transcription(raw_segments):
    return "\n\n".join(
        [
            chunk["speaker"] + " " + tuple_to_string(chunk["timestamp"]) + chunk["text"]
            for chunk in raw_segments
        ]
    )

In [37]:
outputs = pipeline(sample["audio"].copy())

format_as_transcription(outputs)



"SPEAKER_01 (0.0, 15.5) The second and importance is as follows. Sovereignty may be defined to be the right of making laws. In France, the king really exercises a portion of the sovereign power, since the laws have no weight.\n\nSPEAKER_00 (15.5, 21.3) He was in a favored state of mind, owing to the blight his wife's action threatened to cast upon his entire future."

# YEREL DOSYA YOLU

In [ ]:
from pyannote.audio import Pipeline
from speechbox import ASRDiarizationPipeline
import librosa

# Pyannote diarization pipeline'ı Hugging Face token ile yükleme
diarization_pipeline = Pipeline.from_pretrained(
    "pyannote/speaker-diarization@2.1", use_auth_token='HF_TOKEN'
)

# Ses dosyasını yerelden yükleme (elif/e.mp3 dosyasını yüklemek)
audio_path = "/content/recording_2.mp3"
audio_data, sample_rate = librosa.load(audio_path, sr=None)

# ASR ve diarization pipeline'ını birleştirme
pipeline = ASRDiarizationPipeline(
    asr_pipeline=asr_pipeline, diarization_pipeline=diarization_pipeline
)

# Zaman damgalarını yuvarlama ve string formatına dönüştürme fonksiyonu
def tuple_to_string(start_end_tuple, ndigits=1):
    return str((round(start_end_tuple[0], ndigits), round(start_end_tuple[1], ndigits)))

# Segmentleri formatlayarak transkripti oluşturma fonksiyonu
def format_as_transcription(raw_segments):
    return "\n\n".join(
        [
            chunk["speaker"] + " " + tuple_to_string(chunk["timestamp"]) + ": " + chunk["text"]
            for chunk in raw_segments
        ]
    )

# Pipeline'dan çıktı alma (audio_data yerel dosyadan geliyor)
outputs = pipeline(audio_data)

# Çıktıları formatlayıp ekrana yazdırma
transcription = format_as_transcription(outputs)
print(transcription)
